**Chapter 16 – Natural Language Processing with RNNs and Attention**

_This notebook contains all the sample code in chapter 16._

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/jflanigan/handson-ml2/blob/master/16_nlp_with_rnns_and_attention.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

# Setup

First, let's import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures. We also check that Python 3.5 or later is installed (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead), as well as Scikit-Learn ≥0.20 and TensorFlow ≥2.0.

In [10]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    !pip install -q -U tensorflow-addons
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.test.is_gpu_available():
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nlp"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

No GPU was detected. LSTMs and CNNs can be very slow without a GPU.


# Text Classification with RNNs

In [11]:
tf.random.set_seed(42)

You can load the IMDB dataset easily:

In [12]:
(X_train, y_test), (X_valid, y_test) = keras.datasets.imdb.load_data()

In [13]:
X_train[0][:10]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [14]:
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

'<sos> this film was just brilliant casting location scenery story'

In [15]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)

C:\Users\Alison\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Dl Completed...: 100%|██████████| 1/1 [01:45<00:00, 105.68s/ url]


Dataset imdb_reviews downloaded and prepared to C:\Users\Alison\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [16]:
datasets.keys()

dict_keys([Split('train'), Split('test'), Split('unsupervised')])

In [17]:
train_size = info.splits["train"].num_examples
test_size = info.splits["test"].num_examples

In [18]:
train_size, test_size

(25000, 25000)

In [19]:
for X_batch, y_batch in datasets["train"].batch(2).take(1):
    for review, label in zip(X_batch.numpy(), y_batch.numpy()):
        print("Review:", review.decode("utf-8")[:200], "...")
        print("Label:", label, "= Positive" if label else "= Negative")
        print()

Review: This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting  ...
Label: 0 = Negative

Review: I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However  ...
Label: 0 = Negative



In [20]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

In [21]:
preprocess(X_batch, y_batch)

(<tf.Tensor: shape=(2, 53), dtype=string, numpy=
 array([[b'This', b'was', b'an', b'absolutely', b'terrible', b'movie',
         b"Don't", b'be', b'lured', b'in', b'by', b'Christopher',
         b'Walken', b'or', b'Michael', b'Ironside', b'Both', b'are',
         b'great', b'actors', b'but', b'this', b'must', b'simply', b'be',
         b'their', b'worst', b'role', b'in', b'history', b'Even',
         b'their', b'great', b'acting', b'could', b'not', b'redeem',
         b'this', b"movie's", b'ridiculous', b'storyline', b'This',
         b'movie', b'is', b'an', b'early', b'nineties', b'US',
         b'propaganda', b'pi', b'<pad>', b'<pad>', b'<pad>'],
        [b'I', b'have', b'been', b'known', b'to', b'fall', b'asleep',
         b'during', b'films', b'but', b'this', b'is', b'usually', b'due',
         b'to', b'a', b'combination', b'of', b'things', b'including',
         b'really', b'tired', b'being', b'warm', b'and', b'comfortable',
         b'on', b'the', b'sette', b'and', b'having', b'j

In [22]:
from collections import Counter

vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [23]:
vocabulary.most_common()[:3]

[(b'<pad>', 214309), (b'the', 61137), (b'a', 38564)]

In [24]:
len(vocabulary)

53893

In [25]:
vocab_size = 10000
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]]

In [26]:
word_to_id = {word: index for index, word in enumerate(truncated_vocabulary)}
for word in b"This movie was faaaaaantastic".split():
    print(word_to_id.get(word) or vocab_size)

22
12
11
10000


In [27]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [28]:
table.lookup(tf.constant([b"This movie was faaaaaantastic".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   22,    12,    11, 10053]], dtype=int64)>

In [36]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = datasets["train"].repeat().batch(32).map(preprocess)
test_set = datasets["test"].repeat().batch(32).map(preprocess)

train_set = train_set.map(encode_words).prefetch(1)
test_set = test_set.map(encode_words).prefetch(1)

In [30]:
for X_batch, y_batch in train_set.take(1):
    print(X_batch)
    print(y_batch)

tf.Tensor(
[[  22   11   28 ...    0    0    0]
 [   6   21   70 ...    0    0    0]
 [4099 6881    1 ...    0    0    0]
 ...
 [  22   12  118 ...  331 1047    0]
 [1757 4101  451 ...    0    0    0]
 [3365 4392    6 ...    0    0    0]], shape=(32, 60), dtype=int64)
tf.Tensor([0 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 1 1 1 0 1 1 1 1 1 0 0 0 1 0 0 0], shape=(32,), dtype=int64)


In [38]:
embed_size = 16
hidden_dim = 64
dropout = 0.5
nonlinearity = 'tanh'
learning_rate = 1e-3

model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.SimpleRNN(hidden_dim, activation=nonlinearity, dropout=dropout),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, steps_per_epoch=train_size // 32, epochs=20)

Epoch 1/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 13s 12ms/step - accuracy: 0.5035 - loss: 0.6962
Epoch 2/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.5755 - loss: 0.6770
Epoch 3/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.7308 - loss: 0.5421
Epoch 4/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.8090 - loss: 0.4235
Epoch 5/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.8542 - loss: 0.3413
Epoch 6/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.8777 - loss: 0.2927
Epoch 7/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.8909 - loss: 0.2626
Epoch 8/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 12s 15ms/step - accuracy: 0.9084 - loss: 0.2271
Epoch 9/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - accuracy: 0.9149 - loss: 0.2163
Epoch 10/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - accuracy: 0.9259 - loss: 0.1914
Epoch 11/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - accuracy: 0.9295 - loss: 0.1832
Epoch 12/20
781/781 ━━━━━━━━━━━

In [37]:
# Train and test accuracy
print("Train set\n",model.evaluate(train_set, steps=train_size // 32, batch_size=32))

print("Test set\n",model.evaluate(test_set, steps=test_size // 32, batch_size=32))

781/781 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9825 - loss: 0.0664
Train set
 [0.0620143748819828, 0.9839949011802673]
781/781 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7052 - loss: 0.8958
Test set
 [0.9062708616256714, 0.6999439597129822]


## Text Classification with LSTMs

In [39]:
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.LSTM(hidden_dim, activation=nonlinearity, dropout=dropout),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, steps_per_epoch=train_size // 32, epochs=20)

C:\Users\Alison\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 19s 20ms/step - accuracy: 0.6218 - loss: 0.6248
Epoch 2/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 18s 23ms/step - accuracy: 0.8084 - loss: 0.4282
Epoch 3/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 21s 27ms/step - accuracy: 0.8454 - loss: 0.3645
Epoch 4/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 23s 30ms/step - accuracy: 0.8651 - loss: 0.3300
Epoch 5/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 19s 24ms/step - accuracy: 0.8785 - loss: 0.3049
Epoch 6/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 22s 28ms/step - accuracy: 0.8874 - loss: 0.2835
Epoch 7/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 23s 30ms/step - accuracy: 0.8968 - loss: 0.2611
Epoch 8/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - accuracy: 0.9086 - loss: 0.2396
Epoch 9/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 18s 23ms/step - accuracy: 0.9132 - loss: 0.2249
Epoch 10/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 18s 23ms/step - accuracy: 0.9230 - loss: 0.2022
Epoch 11/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 18s 23ms/step - accuracy: 0.9250 - loss: 0.1970
Epoch 12/20
781/781 ━━━━━━━━━━

In [40]:
# Train and test accuracy
print("Train set\n",model.evaluate(train_set, steps=train_size // 32, batch_size=32))

print("Test set\n",model.evaluate(test_set, steps=test_size // 32, batch_size=32))

781/781 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9872 - loss: 0.0543
Train set
 [0.0511460155248642, 0.9878361225128174]
781/781 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7354 - loss: 1.0013
Test set
 [0.9860423803329468, 0.7355954051017761]
